In [28]:
import pandas as pd
import numpy as np

from website import TARGETS, MAIN_CATEGORIES, CATEGORIES, ALGORITHMS, FOLDS, SCORES, GRAPH_SIZE, DOWNLOAD_CONFIG


scores_data = pd.read_feather(f"../data/scores.feather").to_dict()
information_data = pd.read_feather(f"../data/information.feather").to_dict()
targets = ["age"]
examination_categories = ["all"]
laboratory_categories = ["all"]
questionnaire_categories = ["all"]
algorithms = ["elastic_net"]
metric = "r2"

scores = pd.read_feather("../data/scores.feather").set_index(["main_category", "category", "algorithm"])
scores.columns = pd.MultiIndex.from_tuples(
    list(map(eval, scores.columns.tolist())), names=["target", "fold", "metric"]
)

information = pd.read_feather("../data/information.feather").set_index(["main_category", "category"])
information.columns = pd.MultiIndex.from_tuples(
    list(map(eval, information.columns.tolist())), names=["target", "fold", "metric"]
)

In [29]:
scores_grouped_by_categories = scores.reset_index().groupby(by=["main_category", "category"])
best_algorithms = {}
for target in targets:
    best_algorithms[target] = list(map(lambda group: group[1].set_index("algorithm").loc[:, (target, "test", metric)].idxmax(), scores_grouped_by_categories))

In [50]:
best_indexes = pd.MultiIndex.from_tuples(np.dstack(([information.index.get_level_values("main_category").to_list()], [information.index.get_level_values("category").to_list()], [best_algorithms["age"]]))[0])

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [49]:
scores.loc[best_indexes]

target                                                                          age  \
fold                                                                          train   
metric                                                                           r2   
examination   Cardiovascular__Fitness                            elastic_net  0.999   
              Dual__Energy__X-ray__Absorptiometry__-__Femur      elastic_net  0.676   
              Spirometry__-__Pre__and__Post-Bronchodilator       elastic_net  0.620   
              Dual-Energy__X-ray__Absorptiometry__-__Whole__Body elastic_net  0.719   
              Oral__Health                                       elastic_net  0.587   
...                                                                             ...   
questionnaire Arthritis                                          elastic_net  0.020   
              Mental__Health__-__Depression                      elastic_net  0.177   
              Weight__History                                    elastic_net  0.066   
              Alcohol__Use__-__Youth                             elastic_net  0.042   
              Prostate__Specific__Antigen__Follow-up             elastic_net  0.301   

target                                                                               \
fold                                                                                  
metric                                                                       r2_std   
examination   Cardiovascular__Fitness                            elastic_net  0.000   
              Dual__Energy__X-ray__Absorptiometry__-__Femur      elastic_net  0.001   
              Spirometry__-__Pre__and__Post-Bronchodilator       elastic_net  0.002   
              Dual-Energy__X-ray__Absorptiometry__-__Whole__Body elastic_net  0.001   
              Oral__Health                                       elastic_net  0.002   
...                                                                             ...   
questionnaire Arthritis                                          elastic_net  0.002   
              Mental__Health__-__Depression                      elastic_net  0.017   
              Weight__History                                    elastic_net  0.001   
              Alcohol__Use__-__Youth                             elastic_net  0.011   
              Prostate__Specific__Antigen__Follow-up             elastic_net  0.027   

target                                                                               \
fold                                                                           test   
metric                                                                           r2   
examination   Cardiovascular__Fitness                            elastic_net  0.998   
              Dual__Energy__X-ray__Absorptiometry__-__Femur      elastic_net  0.675   
              Spirometry__-__Pre__and__Post-Bronchodilator       elastic_net  0.619   
              Dual-Energy__X-ray__Absorptiometry__-__Whole__Body elastic_net  0.717   
              Oral__Health                                       elastic_net  0.583   
...                                                                             ...   
questionnaire Arthritis                                          elastic_net  0.016   
              Mental__Health__-__Depression                      elastic_net -0.017   
              Weight__History                                    elastic_net  0.066   
              Alcohol__Use__-__Youth                             elastic_net -0.026   
              Prostate__Specific__Antigen__Follow-up             elastic_net -0.054   

target                                                                               \
fold                                                                                  
metric                                                                       r2_std   
examination   Cardiovascular__Fitness                            elastic_net  0.00

In [11]:
np.dstack((scores.index.droplevel("algorithm").drop_duplicates().to_list(), [best_algorithms['age']]))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 303 and the array at index 1 has size 1